## This is gonna be the Flow of my Sales Brochure generator. 

- 1st we get website content and website links - all the links on the landing page - about me, career pages, contact me, blog, projects, plans, associations - whatever links the landing page of the website contains.
- Then I make any llm to detect the important links of these list of links - which can be included in sales brochure - This will be a separate llm call - and it will ONLY send back list of links that are imp.
- Then we will join the content on main landing page. And the imp links given by LLM, and their content in same text.
- This will become the user input later on on our final model.
- Now what we will do is - we will ask our another LLM to check the landing page, and the main links and their content - i.e, related pages of same company/person's website - and return a brochure for prospective customers, investors and recruits.
- Finally we will try to stream it. 

In [1]:
# Ok first lets get the imports - 

In [4]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scrapper import fetch_website_links, fetch_website_contents

In [5]:
#  Now lets get the links - from landing page. 

In [15]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai_obj = OpenAI()

In [6]:
links = fetch_website_links("https://in.indeed.com/?r=us")

In [7]:
links

['https://www.indeed.com/',
 'https://www.indeed.com/',
 'https://www.indeed.com/companies',
 'https://www.indeed.com/career/salaries',
 'https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com',
 'https://www.indeed.com/promo/resume',
 'https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com',
 'https://www.indeed.com/hire?co=US&hl=en&ikw=gnav-header-hire&isid=employerlink-US',
 'https://www.indeed.com/',
 'https://www.indeed.com/companies',
 'https://www.indeed.com/career/salaries',
 'https://www.indeed.com/',
 'https://support.indeed.com/hc/en-us/articles/33465379855501-Troubleshooting-Cloudflare-Errors',
 'https://www.indeed.com/support/contact']

In [8]:
# Now lets make a llm call - but first lets get the system and user prompts? 

In [18]:
system_prompt_links = """
You are an helping assisstant who when recieves a list of links from a company/person's website - you will consider all the links. 
And you will return only the links that are improtant and must be added to the sales brochure created for prospective customers, investors and recruits.
These important links should cover main pages that must be added in sales brochure like about page, contact us, or blog - or main content. 

Reply in JSON format.
Example of a reply - 
Links = [
    {"Type":"About", "Link":"<the link>"},
    {"Type":"Careers", "Link":"<the link>"},
]
"""

In [12]:
def get_user_prompt_for_links(url):
    user_prompt = """
    Here is the list of links from the website. Please go through these and select the main links that can be added to the sales brochure. 
    """
    user_prompt +=  "\n".join(fetch_website_links(url))
    return user_prompt
    

In [13]:
get_user_prompt_for_links("https://in.indeed.com/?r=us")

'\n    Here is the list of links from the website. Please go through these and select the main links that can be added to the sales brochure. \n    https://www.indeed.com/\nhttps://www.indeed.com/\nhttps://www.indeed.com/companies\nhttps://www.indeed.com/career/salaries\nhttps://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com\nhttps://www.indeed.com/promo/resume\nhttps://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com\nhttps://www.indeed.com/hire?co=US&hl=en&ikw=gnav-header-hire&isid=employerlink-US\nhttps://www.indeed.com/\nhttps://www.indeed.com/companies\nhttps://www.indeed.com/career/salaries\nhttps://www.indeed.com/\nhttps://support.indeed.com/hc/en-us/articles/33465379855501-Troubleshooting-Cloudflare-Errors\nhttps://www.indeed.com/support/contact'

In [21]:
def get_imp_links_llm(url):
    message = [
        {"role":"system", "content":system_prompt_links},
        {"role":"user", "content":get_user_prompt_for_links(url)}
    ]
    response = openai_obj.chat.completions.create( model="gpt-5-nano", messages=message, response_format={"type": "json_object"})
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links)} relevant links")
    return links

In [54]:
links2 = get_imp_links_llm("https://in.indeed.com/?r=us")['Links']

KeyboardInterrupt: 

In [39]:
links2 = links2['Links']

In [35]:
links2['Links']

[{'Type': 'Homepage', 'Link': 'https://www.indeed.com/'},
 {'Type': 'For Employers',
  'Link': 'https://www.indeed.com/hire?co=US&hl=en&ikw=gnav-header-hire&isid=employerlink-US'},
 {'Type': 'Companies', 'Link': 'https://www.indeed.com/companies'},
 {'Type': 'Salaries', 'Link': 'https://www.indeed.com/career/salaries'},
 {'Type': 'Resume Promotion', 'Link': 'https://www.indeed.com/promo/resume'},
 {'Type': 'Contact', 'Link': 'https://www.indeed.com/support/contact'}]

In [47]:
for link in links2:
    print(f" Type - {link['Type']}, and URL = {link['Link']}")

 Type - Homepage, and URL = https://www.indeed.com/
 Type - For Employers, and URL = https://www.indeed.com/hire?co=US&hl=en&ikw=gnav-header-hire&isid=employerlink-US
 Type - Companies, and URL = https://www.indeed.com/companies
 Type - Salaries, and URL = https://www.indeed.com/career/salaries
 Type - Resume Promotion, and URL = https://www.indeed.com/promo/resume
 Type - Contact, and URL = https://www.indeed.com/support/contact


In [45]:
for i in range(len(links2)):
    print(f" Type - {links2[i]['Type']}, and URL = {links2[i]['Link']}")

 Type - Homepage, and URL = https://www.indeed.com/
 Type - For Employers, and URL = https://www.indeed.com/hire?co=US&hl=en&ikw=gnav-header-hire&isid=employerlink-US
 Type - Companies, and URL = https://www.indeed.com/companies
 Type - Salaries, and URL = https://www.indeed.com/career/salaries
 Type - Resume Promotion, and URL = https://www.indeed.com/promo/resume
 Type - Contact, and URL = https://www.indeed.com/support/contact


In [59]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = get_imp_links_llm(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['Links']:
        result += f"\n\n### Link: {link['Type']}\n"
        result += fetch_website_contents(link["Link"])
    return result

In [55]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [56]:
def create_brochure(company_name, url):
    response = openai_obj.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [60]:
create_brochure("Indeed", "https://in.indeed.com/?r=us")

Found 1 relevant links


# Indeed: Connecting People to Opportunity

---

## About Indeed

Indeed is a leading global employment website dedicated to helping people find jobs, research companies, and explore salary information. With a comprehensive platform designed for both jobseekers and employers, Indeed simplifies the hiring process and supports career growth worldwide.

---

## Services for Jobseekers

- **Find Jobs:** Access millions of job listings across industries and locations worldwide.
- **Company Reviews:** Read authentic employee reviews to learn about company culture and work environments.
- **Salary Information:** Research salary ranges to make informed career decisions.
- **Resume Upload:** Easily upload and manage your resume to apply quickly to job postings.

---

## Services for Employers

- **Post Jobs:** Effortlessly post job openings to attract qualified candidates.
- **Hiring Resources:** Utilize advanced tools and expert guidance to streamline recruitment.
- **Employer Branding:** Showcase your company culture and values to attract the best talent.

---

## Company Culture

While detailed internal culture specifics are not publicly available, Indeed’s platform emphasizes transparency and empowerment for both jobseekers and employers. Their commitment to an easy-to-navigate, resource-rich environment reflects a customer-focused and user-friendly approach to the world of work.

---

## Careers at Indeed

Information about specific job openings is not provided directly, but as a leading tech-driven company in employment services, Indeed offers opportunities in technology, product development, marketing, and customer support. Interested candidates are encouraged to visit their website or contact support to explore career options.

---

## Contact & Support

**Headquarters:**  
6433 Champion Grandview Way Building 1, Austin, TX 78750

**Support:**  
Visit the Indeed Help Center for FAQs or contact their support team directly for assistance.

---

## Why Choose Indeed?

- Access to millions of jobs globally in one convenient place.
- Trusted company reviews and salary insights for informed career decisions.
- Tools designed to simplify the job search and hiring process.
- Dedicated support and resources tailored to both jobseekers and employers.

---

Discover your next opportunity or find the perfect candidate with Indeed — where opportunity meets talent.  

Visit: [Indeed.com](https://www.indeed.com)

---

© 2026 Indeed, All Rights Reserved

## Lets try streaming as well - 

In [61]:
def create_brochure(company_name, url):
    response = openai_obj.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response_final = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in response:
        response_final += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [63]:
create_brochure("Indeed", "https://in.indeed.com/?r=us")

Found 1 relevant links


TypeError: Markdown expects text, not <openai.Stream object at 0x00000185BADCE900>